In [15]:
import pandas as pd
df=pd.read_csv('stackoverflowtags.csv')
df.head(2)

,title,tags
0,How to draw a stacked dotplot in R?,['r']
1,mysql select all records where a datetime fiel...,"['php', 'mysql']"


In [16]:
df.title.value_counts() 

Conversion failed when converting date and/or time from character string    3
"FormatException was unhandled" Input string was not in a correct format    2
Input string was not in a correct format                                    2
JavaScript runtime error: '$' is undefined                                  2
Object reference not set to an instance of an object                        2
                                                                           ..
NSPredicate predicateWithFormat:(NSString*) inconsistency?                  1
document.location.href is not working in IE                                 1
segmentation fault in C, core dumped, gdb output                            1
C# - are there any events fired right after loading a form?                 1
Create a List of primitive int?                                             1
Name: title, Length: 99984, dtype: int64

In [17]:
df.title=df.title.drop_duplicates(keep='first') 
df.title.value_counts()

How to draw a stacked dotplot in R?                                            1
Hibernate JPA - ManyToOne relationship not populated                           1
how to work treeview nodes contains in asp.net?                                1
Initialize script                                                              1
ASP.NET C# Must declare the scalar variable                                    1
                                                                              ..
Can jquery animate the css background property?                                1
how to access variable from php in jquery using ajax(here i am using Json)     1
Best way to handle session management in spring mvc filters or interceptors    1
Can't run the .java file                                                       1
Create a List of primitive int?                                                1
Name: title, Length: 99984, dtype: int64

In [18]:
import re       



from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer


def clean(s):
    s=str(s)
    s=s.lower()
    html=re.compile('<.*?>')   
    cleaned = re.sub(html,' ',s)
    fil=[]
    for i in cleaned.split():   
        if i!='c++':
            cleaned=re.sub('[^A-Za-z]', '', i) 
            fil.append(cleaned)
        else:
            fil.append(i)
    return fil       

stop=set(stopwords.words('english'))  
sno=SnowballStemmer('english')
clean(df.title[0])              

['how', 'to', 'draw', 'a', 'stacked', 'dotplot', 'in', 'r']

In [19]:


def stem(s):
    fil=[]
    for i in s:
        if i not in stop:
            s=(sno.stem(i).encode('utf8'))  
            fil.append(s)
    s=b' '.join(fil)
    return s

In [20]:

ques=[]
for j in df.title:
    ques.append(stem(clean(j)))
df['cleanQues'] = ques

import re
ctags=[]
for i in df.tags:
    ctags.append(re.sub('[^A-Za-z#+-]', ' ', i)) 
df['cleanTags']=ctags

In [21]:
df.head(10)

,title,tags,cleanQues,cleanTags
0,How to draw a stacked dotplot in R?,['r'],b'draw stack dotplot r',r
1,mysql select all records where a datetime fiel...,"['php', 'mysql']",b'mysql select record datetim field less speci...,php mysql
2,How to terminate windows phone 8.1 app,['c#'],b'termin window phone app',c#
3,get current time in a specific country via jquery,"['javascript', 'jquery']",b'get current time specif countri via jqueri',javascript jquery
4,Configuring Tomcat to Use SSL,['java'],b'configur tomcat use ssl',java
5,Awesome nested set plugin - how to add new chi...,['ruby-on-rails'],b'awesom nest set plugin add new children tre...,ruby-on-rails
6,How to create map from JSON response in Ruby o...,"['ruby', 'ruby-on-rails-3', 'json']",b'creat map json respons rubi rail ',ruby ruby-on-rails- json
7,rspec test if method is called,['ruby'],b'rspec test method call',ruby
8,SpringBoot Catalina LifeCycle Exception,"['java', 'spring', 'spring-mvc']",b'springboot catalina lifecycl except',java spring spring-mvc
9,How to import data from excel to mysql databas...,"['php', 'codeigniter']",b'import data excel mysql databas use php',php codeigniter


In [22]:


d=pd.DataFrame()
d['text']=df.cleanQues
d['tags']=df.cleanTags
d.to_csv('datafinal',index=False)
df = pd.read_csv('datafinal')
df.head()

,text,tags
0,b'draw stack dotplot r',r
1,b'mysql select record datetim field less speci...,php mysql
2,b'termin window phone app',c#
3,b'get current time specif countri via jqueri',javascript jquery
4,b'configur tomcat use ssl',java


In [23]:


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.text, df.tags, test_size=0.2, random_state=9)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tfvectorizer = TfidfVectorizer(min_df=0.00009, max_features=200000, smooth_idf=True, norm="l2",
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
x_train_multilabel = tfvectorizer.fit_transform(x_train)
x_test_multilabel = tfvectorizer.transform(x_test)

vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary=True)
y_train_multilabel = vectorizer.fit_transform(y_train)
y_test_multilabel = vectorizer.transform(y_test)

c:\Users\DHIRAAJ\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [25]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score


classifier = OneVsRestClassifier(SGDClassifier(loss='log_loss', max_iter = 5, tol = None, alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train_multilabel)
predictions = classifier.predict(x_test_multilabel)

print("accuracy :",metrics.accuracy_score(y_test_multilabel,predictions))
print("macro f1 score :",metrics.f1_score(y_test_multilabel, predictions, average = 'macro'))
print("micro f1 scoore :",metrics.f1_score(y_test_multilabel, predictions, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(y_test_multilabel,predictions))

accuracy : 0.33015
macro f1 score : 0.48859792284426407
micro f1 scoore : 0.6387307512832479
hamming loss : 0.011613


In [26]:

print("Precision recall report :\n",metrics.classification_report(y_test_multilabel, predictions))

Precision recall report :
               precision    recall  f1-score   support

           0       0.77      0.48      0.60       327
           1       0.65      0.11      0.19        96
           2       0.92      0.52      0.67       555
           3       0.95      0.72      0.82       294
           4       0.77      0.20      0.32        83
           5       0.56      0.36      0.44       447
           6       0.85      0.39      0.54       950
           7       0.70      0.30      0.42       588
           8       0.84      0.60      0.70      3763
           9       0.89      0.46      0.61      1295
          10       0.32      0.06      0.10        96
          11       0.17      0.01      0.02       101
          12       0.97      0.75      0.85       140
          13       0.66      0.21      0.32       359
          14       0.74      0.63      0.68        87
          15       0.15      0.02      0.04       150
          16       0.48      0.17      0.25       132


c:\Users\DHIRAAJ\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:

import dill
model_data = 'model_data.sav'
tfidf_data = 'tfidf_data.sav'
bow_data = 'bow_data.sav'
dill.dump(classifier, open(model_data, 'wb'))
dill.dump(tfvectorizer, open(tfidf_data, 'wb'))
dill.dump(vectorizer, open(bow_data, 'wb'))

In [28]:
import re   

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
def clean(s):
    s=str(s)
    s=s.lower()
    html=re.compile('<.*?>')   
    cleaned = re.sub(html,' ',s)
    fil=[]
    for i in cleaned.split():     
        if i!='c++':
            cleaned=re.sub('[^A-Za-z]', '', i)     
            fil.append(cleaned)
        else:
            fil.append(i)
    return fil      


stop=set(stopwords.words('english'))    
sno=SnowballStemmer('english')



def stem(s):
    fil=[]
    for _ in s:
        if _ not in stop:
            s=(sno.stem(_).encode('utf8')) 
            fil.append(s)
    s=b' '.join(fil)
    return s

In [37]:
from tkinter import *
from PIL import ImageTk, Image
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import re
import joblib
root = Tk()
# tfvectorizer = TfidfVectorizer()
tfvectorizer = joblib.load("tfidf_data.sav")
classifier = joblib.load("model_data.sav")


root.title("Autonomous Tagging Of Stackoverflow Questions")


img = ImageTk.PhotoImage(Image.open("download (1).png"))   
panel = Label(root, image = img)
panel.image = img
panel.grid(row = 0, column = 0)


img1 = ImageTk.PhotoImage(Image.open("download.png"))
label1 = Label(root,image = img1)
label1.image = img1
label1.grid(row = 1, column = 0)


label2 = Label(root, text = "Enter Your Question",font = "Arial 20", fg = 'black')
label2.grid(row = 2, column = 0)


ques = StringVar()
quesEntered = Entry(root, width = 40, textvariable = ques,font="Arial 18")
quesEntered.grid(column = 0, row = 3, padx = 3, pady = 3)


ans = StringVar()
def pred():
    
    t = ques.get()
    l=[]
    l.append(stem(clean(t)))
    tfvectorizer.fit(l)
    x=tfvectoizer.transform(l)
    t=classifier.predict(x)
    k=vectorizer.inverse_transform(t)
    res = re.sub('[^A-Za-z#+-]+', ' ', str(k[0]))
    ans.set(res)
    label3 = Entry(root,textvariable = ans, font = "Arial 20 bold")
    label3.grid(column =0, row = 5, padx = 3, pady = 3)



button = Button(root,text = "Predict Now", font = "Arial 18", command = pred)
button.grid(column= 0, row = 4, padx = 3, pady = 3)



root.mainloop()


ModuleNotFoundError: No module named '__builtin__'